In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
from ola_models import * 
from ola_RNN import * 

import os, time, copy, math, re, json, pickle, random
import numpy as np
import pandas as pd

import torch, torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
%matplotlib inline 
import seaborn as sns

from functools import partial 

cuda_available = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda_available else "cpu")
print(f'''using device {device}''')

path = !pwd
path = path[0]
print(path)

using device cuda:0
/home/r2/Documents/RNNexp


In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [163]:
a[:,1]

tensor([[2, 2, 2]])

In [212]:
a = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
b = a 
c1 = a[0] @ b[0]
c2 = a[1] @ b[0]
c3 = a[2] @ b[0]
# for i in range(a.shape[0]):            
#     w = a[i].t() @ b[0]
print(c1,c2,c3)
print(a.t() @ b[0])

tensor(14) tensor(32) tensor(50)
tensor([30, 36, 42])


In [213]:
a = a.unsqueeze(0)

In [116]:
import torch.nn.functional as F 

In [238]:
a
# print(sum(a,2))

tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])

In [244]:
torch.sum(a,1)

tensor([[12, 15, 18]])

tensor([[[ 14.,  32.,  50.],
         [ 32.,  77., 122.],
         [ 50., 122., 194.]]], device='cuda:0', dtype=torch.float64)
tensor([[2.3195e-16, 1.5230e-08, 1.0000e+00],
        [8.1940e-40, 2.8625e-20, 1.0000e+00],
        [2.8946e-63, 5.3802e-32, 1.0000e+00]], device='cuda:0',
       dtype=torch.float64)
tensor(1., device='cuda:0', dtype=torch.float64)
torch.Size([3, 3])
torch.Size([3])
tensor([[2.3195e-16, 8.1940e-40, 2.8946e-63],
        [1.5230e-08, 2.8625e-20, 5.3802e-32],
        [1.0000e+00, 1.0000e+00, 1.0000e+00]], device='cuda:0',
       dtype=torch.float64)
tensor(1., device='cuda:0', dtype=torch.float64)


In [260]:
class SelfAttention(nn.Module):
    
    def __init__(self,k,heads=8):        
        super(SAttention,self).__init__()
        self.heads = heads 

        self.Wq = nn.Linear(k, k*heads, bias=False)
        self.Wk = nn.Linear(k, k*heads, bias=False)
        self.Wv = nn.Linear(k, k*heads, bias=False)
                        
        self.merge = nn.Linear(k*heads, k)

    def subsequent_mask(X):
        "Mask out subsequent positions."
        [b,t,k] = X.shape
        attn_shape = (1, t, t)
        subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
        subsequent_mask = torch.from_numpy(subsequent_mask) == 0
        print(subsequent_mask.shape)
        subsequent_mask = torch.expand(b*self.heads,t,t)
        print(subsequent_mask.shape)
        return subsequent_mask
        
    def forward(self,X,mask=None):
        [b, t, k], h = X.shape, self.h 
        
        queries = self.Wq(X).view(b,t,h,k).transpose(1,2)
        keys    = self.Wk(X).view(b,t,h,k).transpose(1,2)        
        values  = self.Wv(X).view(b,t,h,k).transpose(1,2)
        
        queries = queries.contiguous().view(b*h,t,k)
        keys    = keys.contiguous().view(b*h,t,k)
        values  = values.contiguous().view(b*h,t,k)
        
        w_ = torch.bmm(queries, keys.transpose(1,2))
        # w_ will have shape [b*h, t, t] which might not be obvious
        # therefore I believe the mask should have these dimensions as well...         
        if mask is not None: w_ = w_ * mask
        w  = F.softmax(w_,dim=2)

        out = torch.bmm(w,values).view(b,h,t,k).transpose(1,2)
        out = out.contiguous().view(b,t,h*k)
        
        Y = self.merge(out)
        
    def forward_nolearning(self,X):
        w_ = torch.bmm(X, X.transpose(1,2))
        w  = F.softmax(w_, dim=2)    
        Y  = torch.bmm(w, X) 
        return Y 
        

In [ ]:
class TransformerFFN(nn.Module):
    def __init__(self, in_sz, out_sz, d=512):
        self.ln1  = nn.Linear(in_sz,d)
        self.ReLU = nn.ReLU(d)
        self.ln2  = nn.Linear(d,out_sz) 

    def forward(self,x):
        x = self.ln1(x)
        x = self.ReLU(x)
        output = self.ln2(x)
        return output 
    

In [ ]:
class GoogleTransformer(nn.Module):
    def __init__(self, k, heads=8):
        super(GoogleTransformer,self).__init__()
        
        # encoding-block         
        self.SelfAenc = SelfAttention(k, heads)        
        self.enc_ln1  = nn.Linear(2*k,k)
        self.enc_ff   = TransformerFFN(k,k)
        self.enc_ln2  = nn.Linear(2*k,k)        

        # decoding-block         
        self.SelfAdec = SelfAttention(k, heads)        
        self.dec_ln1  = nn.Linear(2*k,k)
        self.dec_ff   = TransformerFFN(k,k)
        self.dec_ln2  = nn.Linear(2*k,k)                
        
    def forward(self,X,Y):
        # encoding-block        
        Xsa  = self.SelfAenc.forward(X)        
        Xln  = self.enc_ln1(torch.cat((X, Xsa),1))        
        Xff  = self.enc_ff(Xln)        
        Xenc = self.enc_ln2(torch.cat((Xln, Xff),1))
        
        # decoding-block         
        mask = self.SelfAdec.subsequent_mask(X)      
        Ysa  = self.SelfAdec.forward(Y,mask)
        Yln1  = self.layernorm_dec1(torch.cat((Y, Ysa),1))
        Yff   = self.ff_dec1(Yln)
        
        
        

In [269]:
[b,t,k] = a.shape

In [ ]:
SA = SelfAttention(k)